# Akeed Restaurant Recommendation Challenge

## Importing libraries:

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import psycopg2
import geopandas as gpd

## Inspecting the base layers:

### Base Layer 1

In [ ]:
#Training Locations
training_locations = pd.read_csv("./train_locations.csv")
training_locations.head()

### Base Layer 2

In [ ]:
#Training Customers
training_customers = pd.read_csv("./train_customers.csv")
training_customers.head()

### Base Layer 3

In [ ]:
#Orders
orders = pd.read_csv("./orders.csv")
orders.head()

### Base Layer 4

In [ ]:
#Vendors
vendors = pd.read_csv("./vendors.csv")
vendors.head()

## Pre-process tables where need be

### Deal with null values

In [ ]:
#Training Locations


In [ ]:
#Orders table


### Deal with mixed data-type fields

In [ ]:
#Orders table


### Scale field values where needed

### Add valuable calculated fields

In [ ]:
#Add geometry field to the training locations table


## Exploring Relationships using plots

### Exploration 1

In [ ]:
# Spatial Relations

#ax = countries.plot(edgecolor = 'grey', facecolor = 'none', figsize = (18, 12), legend=True)
#roads.plot(ax=ax, color = 'purple')
#ax.set(xlim=(15, 35), ylim=(-37.5, -20))
#ax.set_axis_off()
#plt.title(label = 'World Countries', fontweight = 'bold')
#plt.tight_layout()

### Exploration 2

In [ ]:
# Linear Relations



## Prediction Modelling